https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import ee

ee.Initialize()

In [4]:
from birdcall_distribution import geo
import json
from shapely.geometry import mapping

ca_shape = geo.get_shape_us_state("California")
grid = geo.generate_grid(ca_shape, (-124, -114, 32, 42), (0.25, 0.25))

In [5]:
# Import the MODIS land cover collection.
lc = ee.ImageCollection("MODIS/006/MCD12Q1")

# Import the MODIS land surface temperature collection.
lst = ee.ImageCollection("MODIS/006/MOD11A1")

# Import the USGS ground elevation image.
elv = ee.Image("USGS/SRTMGL1_003")

# Initial date of interest (inclusive).
# Final date of interest (exclusive).
i_date = "2017-01-01"
f_date = "2020-01-01"

# temperature day and night, with quality control bands
lst = lst.select("LST_Day_1km", "LST_Night_1km", "QC_Day", "QC_Night").filterDate(
    i_date, f_date
)

## get stats for a single polygon

In [20]:
polygons = [ee.Geometry.Polygon(mapping(p)["coordinates"]) for p in grid.values()]
polygon = polygons[0]

In [21]:
# scale in meters
scale = 1000

# elevation data
elv.sample(polygon, scale).aggregate_stats("elevation").getInfo()

{'max': 122,
 'mean': 1.6581632653061225,
 'min': 0,
 'sample_sd': 11.88140565983313,
 'sample_var': 141.16780045351473,
 'sum': 1300,
 'sum_sq': 112690,
 'total_count': 784,
 'total_sd': 11.873825814940545,
 'total_var': 140.98773948354852,
 'valid_count': 784,
 'weight_sum': 784,
 'weighted_sum': 1300}

In [34]:
elv.reduceRegion(ee.Reducer.percentile([5, 50, 95]), polygon, scale).getInfo()

{'elevation_p5': 0, 'elevation_p50': 0, 'elevation_p95': 0}

In [23]:
# LST collection (land surface temperature)
for property in ["LST_Day_1km", "LST_Night_1km"]:
    temp = lst.mean().sample(polygon, scale).aggregate_mean(property).getInfo()
    print(property, temp)

LST_Day_1km 14386.504953502432
LST_Night_1km 14071.905977195267


In [33]:
# get percentiles
lst.mean().reduceRegion(ee.Reducer.percentile([5, 50, 95]), polygon, scale).getInfo()

{'LST_Day_1km_p5': 14303.942307692309,
 'LST_Day_1km_p50': 14377.311504424779,
 'LST_Day_1km_p95': 14464.238185255199,
 'LST_Night_1km_p5': 14040.665254237289,
 'LST_Night_1km_p50': 14072.679207920792,
 'LST_Night_1km_p95': 14104.058219178081,
 'QC_Day_p5': 3,
 'QC_Day_p50': 3,
 'QC_Day_p95': 3,
 'QC_Night_p5': 3,
 'QC_Night_p50': 3,
 'QC_Night_p95': 3}

In [9]:
lc_point = lc.first().sample(polygon, scale).first().getInfo()
lc_point

{'type': 'Feature',
 'geometry': None,
 'id': '0',
 'properties': {'LC_Prop1': 3,
  'LC_Prop1_Assessment': 99,
  'LC_Prop2': 3,
  'LC_Prop2_Assessment': 99,
  'LC_Prop3': 3,
  'LC_Prop3_Assessment': 99,
  'LC_Type1': 17,
  'LC_Type2': 0,
  'LC_Type3': 0,
  'LC_Type4': 0,
  'LC_Type5': 0,
  'LW': 1,
  'QC': 3}}

In [10]:
lc.select("LC_Type1").first().sample(polygon, scale).aggregate_histogram(
    "LC_Type1"
).getInfo()

{'1': 7, '11': 2, '17': 591, '2': 3, '5': 1, '7': 1}